## Pathing Testbed

In [ ]:
# Change path as needed
MAP_PATH = "../tests/pickle_data/AncientCisternAIE.xz"

In [ ]:
# load cell magic things
%matplotlib notebook
%load_ext line_profiler
%load_ext Cython

In [ ]:
# imports
from matplotlib.backend_bases import MouseButton
import matplotlib.pyplot as plt
import numpy as np

from sc2.ids.unit_typeid import UnitTypeId
from sc2.bot_ai import BotAI
from sc2.position import Point2
from sc2.dicts.unit_trained_from import UNIT_TRAINED_FROM
from sc2.game_info import Race
from sc2.unit import Unit
from sc2.units import Units

from cython_extensions import cy_dijkstra

from tests.load_bot_from_pickle import get_map_specific_bot

In [ ]:
# setup a burnysc2 BOTAI instance we can test with
bot: BotAI = get_map_specific_bot(MAP_PATH)

In [ ]:
cost = np.where(bot.game_info.pathing_grid.data_numpy.T == 1, 1.0, np.inf)
targets_test = np.array([u.position.rounded for u in bot.units])

In [ ]:
%timeit cy_dijkstra(cost, targets_test).get_path(bot.enemy_start_locations[0])

In [ ]:
pathing = cy_dijkstra(cost, targets_test)
pathing.get_path(bot.enemy_start_locations[0], 3)

In [ ]:
%timeit pathing.get_path(bot.enemy_start_locations[0])

In [ ]:
%timeit pathing.get_path(bot.enemy_start_locations[0], 3)

In [ ]:
pathing = None
targets = set()
source = 0, 0

fig, ax = plt.subplots()
ax.set_title("Leftclick to add target. Rightclick to reset.")
img = ax.imshow(cost, picker=True)
fig.colorbar(img)

def draw():
    if pathing is None:
        data = cost
    else:
        data = cost.copy()
        for p in pathing.get_path(source):
            data[p] = np.nan
        for t in targets:
            data[t] = np.nan
    img.set_array(data)
    img.set_clim(vmax=np.ma.masked_invalid(data).max())
    fig.canvas.flush_events()
    
def pick_target(event):
    global pathing
    p = int(event.ydata), int(event.xdata)
    if event.button is MouseButton.LEFT:
        if cost[p] < np.inf:
            targets.add(p)
        pathing = cy_dijkstra(cost, np.array(list(targets)))
    elif event.button is MouseButton.RIGHT:
        targets.clear()
        pathing = None
    draw()

def pick_source(event):
    global source
    source = int(event.ydata), int(event.xdata)
    draw()

fig.canvas.mpl_connect('button_press_event', pick_target)
fig.canvas.mpl_connect('motion_notify_event', pick_source)
plt.show()